# Project 2 Djikstra's Algorithm

(a) Suppose the input graph G = (V, E) is stored in an adjacency matrix and we
use an array for the priority queue. 

Implement the Dijkstra’s algorithm using this
setting and analyze its time complexity with respect to |V| and |E| both
theoretically and empirically.

In [2]:
# Djikstra for part 1a
# Assume it is stored in an adj matrix, assuming V nodes, size of adj matrix is V x V

def dijkstra_1a(adj_matrix, source):
    # measures distance from that 
    V = len(adj_matrix)
    distances = [float('inf') for i in range(V)] 
    
    # distance from root to itself is 0
    distances[source] = 0
    visited = set()
    while len(visited) != V:
        # first we extract cheapest node
        min_node = -1
        distance = float('inf')
        for i in range(V):
            if i not in visited and distances[i] < distance:
                min_node = i
                distance = distances[i]
        
        if min_node == -1:
            return distances
        
        visited.add(min_node)
        for i in range(V):
            if adj_matrix[min_node][i] != 0 and i not in visited:
                distances[i] = min(distances[i], distances[min_node] + adj_matrix[min_node][i])
    return distances
    
graph = [
    [0, 4, 0, 0, 0, 0, 0, 8, 0],
    [4, 0, 8, 0, 0, 0, 0, 11, 0],
    [0, 8, 0, 7, 0, 4, 0, 0, 2],
    [0, 0, 7, 0, 9, 14, 0, 0, 0],
    [0, 0, 0, 9, 0, 10, 0, 0, 0],
    [0, 0, 4, 14, 10, 0, 2, 0, 0],
    [0, 0, 0, 0, 0, 2, 0, 1, 6],
    [8, 11, 0, 0, 0, 0, 1, 0, 7],
    [0, 0, 2, 0, 0, 0, 6, 7, 0]
]

source = 0
print(dijkstra_1a(graph, source))
    
    
    
    # returns the shortest path from node 0 to all other nodes

[0, 4, 12, 19, 21, 11, 9, 8, 14]


In [8]:
# theoretical time complexity is clearly O(V^2) as each time we extract the cheapest node, we add the node to visited and check the rest of the nodes
# and we need to do this V times
# and each time, we look through the array of distances of size V
import random
import time
# empirical time complexity is as follows
sizes = [10 ** i for i in range(1, 5)]



for size in sizes:
    graph = runtimes = [[random.randint(0, 100) for _ in range(size)] for _ in range(size)]
    source = random.randint(0, size - 1)
    start_time = time.perf_counter()
    dijkstra_1a(graph, source)
    end_time = time.perf_counter()
    runtimes.append(end_time - start_time)
    




In [11]:
import seaborn as sb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# plot runtimes aginst sizes

sb.lineplot(x=sizes, y=runtimes)

ValueError: All arrays must be of the same length

In [ ]:
import heapq
def djikstra_1b(adj_list, source):
    pq = []
    heapq.heapify(pq)
    heapq.heappush(pq, (0, source))
    distances = [float('inf') for i in range(len(adj_list))]
    distances[source] = 0
    while len(pq) != 0:
        distance, node = heapq.heappop(pq)
        for neighbour, weight in adj_list[node]:
            if distances[neighbour] > distance + weight:
    